# Advanced DL and RL: Домашнее задание 1

Студент: Самохвалов Михаил

Группа: MADE-DS-32

Email: sam2051@yandex.ru

Первое ДЗ связано с обучением с подкреплением, и оно придумано для ситуации, когда нейронные сети ещё не нужны, и пространство состояний в целом достаточно маленькое, чтобы можно было обучить хорошую стратегию методами TD-обучения или другими методами обучения с подкреплением. Задание получилось, надеюсь, интересное, но в том числе и достаточно техническое, так что для решения придётся немножко попрограммировать. Поэтому в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Любые комментарии, новые идеи и рассуждения на тему, как всегда, категорически приветствуются.


## Часть первая, с блекджеком и стратегиями

Мы будем обучаться играть в очень простую, но всё-таки знаменитую и популярную игру: блекджек. Правила блекджека достаточно просты; давайте начнём с самой базовой версии, которая реализована в OpenAI Gym:
 - численные значения карт равны от 2 до 10 для карт от двойки до десятки, 10 для валетов, дам и королей;
 - туз считается за 11 очков, если общая сумма карт на руке при этом не превосходит 21 (по-английски в этом случае говорят, что на руке есть usable ace), и за 1 очко, если превосходит;
 - игроку раздаются две карты, дилеру — одна в открытую и одна в закрытую;
 - игрок может совершать одно из двух действий:
    - hit  — взять ещё одну карту;
    - stand — не брать больше карт;
 - если сумма очков у игрока на руках больше 21, он проигрывает (bust);
 - если игрок выбирает stand с суммой не больше 21, дилер добирает карты, пока сумма карт в его руке меньше 17;
 - после этого игрок выигрывает, если дилер либо превышает 21, либо получает сумму очков меньше, чем сумма очков у игрока; при равенстве очков объявляется ничья (ставка возвращается);
 - в исходных правилах есть ещё дополнительный бонус за natural blackjack: если игрок набирает 21 очко с раздачи, двумя картами, он выигрывает не +1, а +1.5 (полторы ставки).
 
Именно этот простейший вариант блекджека реализован в OpenAI Gym:
https://github.com/openai/gym/blob/38a1f630dc9815a567aaf299ae5844c8f8b9a6fa/gym/envs/toy_text/blackjack.py

In [174]:
import gym
import numpy as np
from collections import defaultdict
from tqdm.notebook import tqdm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [141]:
env = gym.make('Blackjack-v0', natural=True)
observation = env.reset()
env.seed(42)
# The observation of a 3-tuple of: the players current sum,
#     the dealer's one showing card (1-10 where 1 is ace),
#     and whether or not the player holds a usable ace (0 or 1) - i.e. use Ace as 11.

observation, reward, done, info = env.step(0)

In [143]:
def play_round(strategy):
    obs = env.reset()
    done = False
    while not done:
        action = strategy(obs)
        obs, reward, done, info = env.step(action)
    return reward


### 1. Рассмотрим очень простую стратегию: говорить stand, если у нас на руках комбинация в 19, 20 или 21 очко, во всех остальных случаях говорить hit. Используйте методы Монте-Карло, чтобы оценить выигрыш от этой стратегии.

In [144]:
# hit = 1
# stand = 0

def trivial_strategy(observation):
    if observation[0] > 18:
        return 0
    else:
        return 1

In [152]:
n_iter = 100_000
mean_reward = 0

for i in tqdm(range(1, n_iter+1)):
    reward = play_round(trivial_strategy)
    mean_reward += reward / n_iter
#     mean_reward += (reward - mean_reward) / i
env.close()

In [153]:
print(f'Средняя награда при {n_iter} сыгранных играх {mean_reward:.3f}')
print(f'Средняя награда при {n_iter} сыгранных играх без natural 0.20')

Средняя награда при 100000 сыгранных играх -0.179
Средняя награда при 100000 сыгранных играх без natural 0.20


### 2. Реализуйте метод обучения с подкреплением без модели (можно Q-обучение, но рекомендую попробовать и другие, например Monte Carlo control) для обучения стратегии в блекджеке, используя окружение Blackjack-v0 из OpenAI Gym.

In [192]:
class QLearning():
    
    def __init__(self, lr, dr, eps, plot_step=50):
        self.lr = lr # learning rate
        self.dr = dr # discount rate
        self.eps = eps
        
        self.plt_step = plot_step
        
        self.env = gym.make('Blackjack-v0', natural=True)
#         env.seed(42)
        self.Q = defaultdict(lambda: [0.0]*env.action_space.n) # or we can random init here!
    
    def get_action(self, state):
        if np.random.rand() < self.eps:
            action = np.random.choice(self.env.action_space.n)
        else:
            action = np.argmax(self.Q[state])
        return action
    
    def update_Q(self, state, action, reward):
        self.Q[state][action] = self.Q[state][action] + \
                 self.lr * (reward + self.dr * np.max(self.Q[state]) - self.Q[state][action])
    
    def play_game(self):
        state = self.env.reset()
        done = False
        steps = []
        while not done:
            action = self.get_action(state)
            steps.append((state, action))
            state, reward, done, _ = self.env.step(action)
        return steps, reward
    
    def run(self, n_iter, train=True, plot=False):
        mean_reward = 0
        plot_rewards = []
        for i in tqdm(range(n_iter)):
            steps, reward = self.play_game()
            mean_reward += (reward - mean_reward) / (i + 1)
            if i % self.plot
            if train:
                for state, action in steps[::-1]:
                    self.update_Q(state, action, reward)
        return mean_reward
        

In [196]:
model = QLearning(0.0002, 1.0, 0.2)
train_mean_reward = model.run(10000, train=True)
test_mean_reward = model.run(10000, train=False)

print('Final train mean reward', train_mean_reward)
print('Final test mean reward', test_mean_reward)


Final train mean reward -0.16344999999999982
Final test mean reward -0.12524999999999992


In [173]:
Q = defaultdict(lambda: [0]*env.action_space.n)
Q[3]

[0, 0]